# Integrated Bioinformatics Project : Flux_Balance_F_Prausnitzii 
#### Students: Michael Shawn Neilsson, Biancamaria FLorenzi, Prabhat Juyal, Tim Blokker
#### Supervisors: Clémence Joseph, Karoline Faust

In [78]:
#import packages
import io
import cobra
import functions
from functions import *

In [79]:
#model = cobra.io.read_sbml_model('Data/f_prausnitzii.xml')
#info(model)

In [80]:
model=cobra.io.load_matlab_model('Data/iFpraus_v_1_0.mat')
info(model)

Metabolites :  833
Reactions :  1030
Genes :  602
Compartments :  {'c': '', 'e': ''}
Objective function : 
 Maximize
1.0*Biomass_FP - 1.0*Biomass_FP_reverse_ee33b 



In [81]:
#https://www.thielelab.eu/in-silico-models
#model = cobra.io.read_sbml_model('Data/iFpraus_v_1_0.xml')
#(model2,errors)=(cobra.io.sbml.validate_sbml_model('Data/iFpraus_v_1_0.xml'))
#print(errors)

In [82]:
outputmodel= cobra.io.save_json_model(model, "Data/iFpraus.json")

In [83]:
#use for later perhaps 
#modelMedium = setMediumConstraints(starting_model, set_inf,
#current_inf,... medium_composition, met_Conc_mM, cellConc, t, cell-
#Weight, mediumCompounds,... mediumCompounds_lb);

### What are the different reactions
e: extracellular

c: cytosol

EX_ : exchange reaction
### Gifu Anaerobic Medium (mGAM) from:
https://hyserve.com/files/05433_GAM-Broth_Modified_final.pdf


- Peptone, Soya Peptone, Proteose Peptone: source of amino acids -> most likely all of them 
- Yeast extract, liver extract, meat extract, digested serum -> amino acids but also a lot of other crap, will leave this till the end 
- Dextrose -> same as glucose (https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:17634) and so -> glc_D
- Soluble starch -> starch1200 (potatoe starch)
- L-Tryptophane -> trp__L
- L-Cystein Hydrochloride -> cys_L and cl and h (hydrochloride<-> HCL becomes cl and h)
- Sodium Thioglycolate -> thiog and na1
- L-Arginine -> arg_L
- Vitamine K1 -> phllqne https://pubchem.ncbi.nlm.nih.gov/compound/Vitamin-K1#section=Depositor-Supplied-Synonyms 
- Hemin -> Iron and Chlorine  fe3 and cl  and perhaps ppp9 https://pubchem.ncbi.nlm.nih.gov/#query=3-%5B18-(2-carboxyethyl)-8%2C13-bis(ethenyl)-3%2C7%2C12%2C17-tetramethylporphyrin-21%2C23-diid-2-yl%5Dpropanoic%20acid%3Biron(3%2B)%3Bchloride 
- Potassium Dihydrogen Phosphate -> pi, k, h
- Sodium Chloride -> na1 cl

Digested serum : https://www.nebiolabs.com.au/-/media/catalog/datacards-or-manuals/p8108datasheet-lot0021306.pdf	

Generally useful page ; https://pubchem.ncbi.nlm.nih.gov/#query=Hemin

check also: https://opencobra.github.io/cobratoolbox/latest/tutorials/tutorialMetabotoolsI.html 

In [84]:
import numpy as np
aa=("ala,arg,asn,asp,asx,cys,glu,gln,glx,gly,his,ile,leu,lys,met,phe,pro,ser,thr,trp,tyr,val").split(",")
aa=[aminoacid +"_L(e)" for aminoacid in aa]
print(aa)

['ala_L(e)', 'arg_L(e)', 'asn_L(e)', 'asp_L(e)', 'asx_L(e)', 'cys_L(e)', 'glu_L(e)', 'gln_L(e)', 'glx_L(e)', 'gly_L(e)', 'his_L(e)', 'ile_L(e)', 'leu_L(e)', 'lys_L(e)', 'met_L(e)', 'phe_L(e)', 'pro_L(e)', 'ser_L(e)', 'thr_L(e)', 'trp_L(e)', 'tyr_L(e)', 'val_L(e)']


In [111]:
model1=model.copy()
print("\nModel Medium")
#model1.objective = {model1.reactions.get_by_id('EX_ac(e)'): 1}
model1.objective = {model1.reactions.get_by_id('Biomass_FP'): 1}

print(model1.summary())


Model Medium
Objective
1.0 Biomass_FP = 69.87885590787751

Uptake
------
    Metabolite          Reaction   Flux  C-Number  C-Flux
    Lcystin[e]     EX_Lcystin(e)  3.502         6   0.05%
        ade[e]         EX_ade(e)  864.9         5  10.85%
     adocbl[e]      EX_adocbl(e) 0.3929        72   0.07%
     alaala[e]      EX_alaala(e)  6.462         6   0.10%
     alagln[e]      EX_alagln(e)  30.81         8   0.62%
      arg-L[e]       EX_arg_L(e)   1000         6  15.05%
      asn-L[e]       EX_asn_L(e)   16.4         4   0.16%
        btn[e]         EX_btn(e) 0.3929        10   0.01%
        ca2[e]         EX_ca2(e) 0.3929         0   0.00%
         cl[e]          EX_cl(e) 0.3929         0   0.00%
    cobalt2[e]     EX_cobalt2(e) 0.3929         0   0.00%
     cps_fp[e]      EX_cps_fp(e) 0.3929        63   0.06%
        cu2[e]         EX_cu2(e) 0.3929         0   0.00%
        fe3[e]         EX_fe3(e) 0.7858         0   0.00%
        fol[e]         EX_fol(e)  1.179        19   0.06

In [86]:
#met_Conc_mM = [0.1;1.15;0.15;0.379;0.208;2;0.136;0.133;0.0968;0.382;0.382;0.274;0.101;0.0909;0.153;0.174;...
#0.286;0.168;0.0245;0.129;0.171;0.00863;0.00082;0.0214;0.000524;0.00227;0.082;0.00485;0.000532;0.00297;...
#0.194;0.424;0;5.33;23.81;127.26;5.63;11.11;0;0;0;1;0;0.00326;0.0073]
#customizedConstraints_lb = [-2.3460;0;0;-500;-100;-100;-100];
#customizedConstraints_ub = [500;0;0;500;500;500;500];

In [113]:
MediumMgam = ['ala_L_(e)', 'arg_L_(e)', 'asn_L_(e)', 'asp_L_(e)', 'asx_L_(e)', 'cys_L_(e)', 'glu_L_(e)', 'gln_L_(e)', 
              'glx_L_(e)', 'gly_L_(e)', 'his_L_(e)', 'ile_L_(e)', 'leu_L_(e)', 'lys_L_(e)', 'met_L_(e)', 'phe_L_(e)', 
              'pro_L_(e)', 'ser_L_(e)', 'thr_L_(e)', 'trp_L_(e)', 'tyr_L_(e)', 'val_L_(e)', 'glc_D_(e)', 'starch1200_(e)', 
              'cl_(e)', 'h_(e)',"ac_(e)", "so4_(e)" #'thiog_(e)'
              , 'na1_(e)', 'fe3_(e)', 'ppp9_(e)', 'h2o_(e)', 
              'pi_(e)', 'k_(e)']
MediumMgam=[mets[:-4] +"(e)" for mets in MediumMgam]
print(MediumMgam)
model2=model.copy()
model_mgam=medium(model2,MediumMgam)
print('\nMedium_mgam')
model_mgam.objective = {model_mgam.reactions.get_by_id('Biomass_FP'): 1}
print(model_mgam.summary())

['ala_L(e)', 'arg_L(e)', 'asn_L(e)', 'asp_L(e)', 'asx_L(e)', 'cys_L(e)', 'glu_L(e)', 'gln_L(e)', 'glx_L(e)', 'gly_L(e)', 'his_L(e)', 'ile_L(e)', 'leu_L(e)', 'lys_L(e)', 'met_L(e)', 'phe_L(e)', 'pro_L(e)', 'ser_L(e)', 'thr_L(e)', 'trp_L(e)', 'tyr_L(e)', 'val_L(e)', 'glc_D(e)', 'starch1200(e)', 'cl(e)', 'h(e)', 'ac(e)', 'so4(e)', 'na1(e)', 'fe3(e)', 'ppp9(e)', 'h2o(e)', 'pi(e)', 'k(e)']

Medium_mgam
Objective
1.0 Biomass_FP = 6.528622762307612e-11

Uptake
------
Metabolite   Reaction  Flux  C-Number   C-Flux
    na1[e]  EX_na1(e) 33.47         0    0.00%
   hpyr[c]  sink_hpyr  0.05         3  100.00%

Secretion
---------
Metabolite   Reaction   Flux  C-Number  C-Flux
    na1[c]     DM_NA1 -33.47         0   0.00%
     ac[e]   EX_ac(e)  -0.05         2  66.67%
    co2[e]  EX_co2(e)  -0.05         1  33.33%



In [117]:
MediumMgam = ['ala_L_(e)', 'arg_L_(e)', 'asn_L_(e)', 'asp_L_(e)', 'asx_L_(e)', 'cys_L_(e)', 'glu_L_(e)', 'gln_L_(e)', 
              'glx_L_(e)', 'gly_L_(e)', 'his_L_(e)', 'ile_L_(e)', 'leu_L_(e)', 'lys_L_(e)', 'met_L_(e)', 'phe_L_(e)', 
              'pro_L_(e)', 'ser_L_(e)', 'thr_L_(e)', 'trp_L_(e)', 'tyr_L_(e)', 'val_L_(e)', 'glc_D_(e)', 'starch1200_(e)', 
              'cl_(e)', 'h_(e)',"ac_(e)", "so4_(e)" #'thiog_(e)'
              , 'na1_(e)', 'fe3_(e)', 'ppp9_(e)', 'h2o_(e)', 
              'pi_(e)', 'k_(e)']
MediumMgam=[mets[:-4] +"(e)" for mets in MediumMgam]
print(MediumMgam)
model2=model.copy()
model_mgam=medium(model2,MediumMgam)
print('\nMedium_mgam')
for i in range(1,len(model_mgam.reactions)):
    model_mgam.reactions[i].upper_bound=1000
    model_mgam.reactions[i].lower_bound=-1000 #does not change till 77,78
model_mgam.objective = {model_mgam.reactions.get_by_id('Biomass_FP'): 1}
print(model_mgam.summary())

['ala_L(e)', 'arg_L(e)', 'asn_L(e)', 'asp_L(e)', 'asx_L(e)', 'cys_L(e)', 'glu_L(e)', 'gln_L(e)', 'glx_L(e)', 'gly_L(e)', 'his_L(e)', 'ile_L(e)', 'leu_L(e)', 'lys_L(e)', 'met_L(e)', 'phe_L(e)', 'pro_L(e)', 'ser_L(e)', 'thr_L(e)', 'trp_L(e)', 'tyr_L(e)', 'val_L(e)', 'glc_D(e)', 'starch1200(e)', 'cl(e)', 'h(e)', 'ac(e)', 'so4(e)', 'na1(e)', 'fe3(e)', 'ppp9(e)', 'h2o(e)', 'pi(e)', 'k(e)']

Medium_mgam
Objective
1.0 Biomass_FP = 131.5139863809376

Uptake
------
 Metabolite       Reaction   Flux  C-Number  C-Flux
    4hba[c]        DM_4HBA  64.77         7   4.63%
    3mop[e]     EX_3mop(e)  521.6         6  31.94%
   acglu[e]    EX_acglu(e)  57.79         7   4.13%
     ade[e]      EX_ade(e)  28.42         5   1.45%
  adocbl[e]   EX_adocbl(e) 0.7395        72   0.54%
  alaala[e]   EX_alaala(e)  38.38         6   2.35%
  alahis[e]   EX_alahis(e)  12.18         9   1.12%
  alaleu[e]   EX_alaleu(e)  57.69         9   5.30%
   arg-L[e]    EX_arg_L(e)  37.91         6   2.32%
     btn[e]      EX

In [115]:
fba = model.optimize()
fluxes = deleteNull(fba.fluxes)
print('\nActive reaction : %i which represent %f%% of the model' %(len(fluxes),len(fluxes)/len(model.reactions)*100))
print(fba.objective_value)


Active reaction : 364 which represent 35.339806% of the model
69.8788559078775


In [116]:
fba_model_mgam=model_mgam.optimize()
fluxes_model_mgam = deleteNull(fba_model_mgam.fluxes)
print('\nActive reaction : %i which represent %f%% of the model' %(len(fluxes_model_mgam),len(fluxes_model_mgam)/len(model_mgam.reactions)*100))
print(fba_model_mgam.objective_value)


Active reaction : 709 which represent 68.834951% of the model
131.5139863809377
